In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection  import train_test_split
from sklearn.metrics import r2_score

In [2]:
#Import Dataset
df = pd.read_csv("./Data/cleaned_data.csv", delimiter = '\t')
df

,Player,Tm,PTS,Age,G,GS,MP,FG,FGA,FG%,...,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF
0,Precious Achiuwa,NYK,5.2,-0.635196,0.206292,-0.315059,0.032834,-0.265682,-0.331798,0.435515,...,-0.136166,-0.539862,1.715319,-0.366901,0.246799,-0.605531,-0.277504,1.987459,-0.591965,0.521759
1,Bam Adebayo,MIA,22.6,-0.195287,-0.526429,0.200930,1.376889,2.154425,2.038275,0.473375,...,1.154829,0.363192,1.496885,1.984685,1.998319,0.998600,-0.277504,-0.830641,0.663333,1.215277
2,Nickeil Alexander-Walker,MIN,7.3,-0.415242,1.488553,-0.487055,0.271777,-0.166225,0.025336,-0.224329,...,-0.505022,1.148218,-0.578238,-0.541093,-0.561594,0.246663,0.123752,0.036467,-0.068924,-0.085069
3,Grayson Allen,PHO,3.5,0.244623,-1.075970,-0.315059,0.114970,-0.862421,-0.705166,-1.122148,...,0.140475,1.148218,0.186281,0.155674,0.179433,-0.405015,-0.076876,-0.830641,0.558725,0.348380
4,Jarrett Allen,CLE,17.0,-0.415242,-0.709609,0.028934,0.884069,1.060404,0.398704,1.452323,...,2.215289,0.302805,3.244357,3.204025,3.480375,-0.254628,1.528145,1.337129,0.349509,0.348380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Jaylin Williams,OKC,4.4,-1.295060,0.389472,-0.659052,-0.542123,-0.497747,-0.575299,0.419289,...,-0.597236,0.462006,-0.469021,0.068578,-0.090031,-0.154370,-0.277504,-0.180310,-0.696573,-0.431829
210,Kenrich Williams,OKC,0.6,0.464577,-0.160069,-0.659052,-1.146948,-0.928725,-0.932433,-0.851721,...,-0.781664,-1.596628,-0.796672,-0.715284,-0.797376,-0.705790,-0.879387,-0.830641,-1.010398,-1.038657
211,Delon Wright,MIA,8.0,0.904487,-0.709609,-0.487055,0.510720,-0.033617,-0.299331,1.041273,...,-0.551129,1.148218,-0.359804,-0.062066,-0.157398,-0.003982,0.926262,-0.180310,-1.010398,-0.258449
212,Thaddeus Young,PHO,0.0,1.784306,-1.259150,-0.659052,-1.191750,-1.028181,-1.111000,-2.203859,...,-0.781664,-1.596628,-0.905889,-1.150763,-1.167890,-0.906306,-1.080014,-0.830641,-1.010398,-1.385416


### First, we will do a multi-feature linear regression model for the entire dataset

In [9]:
#Separating Features from Target
X = df.iloc[:, 3:]
y = df['PTS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10271970)

reg = LinearRegression()
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)
corr = r2_score(y_test, y_pred)
print(f"r^2 value of all features for all players: {corr}")

r^2 value of all features for all players: 0.9999196365482985


### Now, we will do the multi-feature linear regression per team

In [10]:
y_pred[10]

2.419583825921764

In [ ]:
y_test.to_list

<bound method IndexOpsMixin.tolist of 30      8.1
170     0.0
58     14.3
177     1.4
202    13.0
       ... 
142     6.3
0       5.2
180     2.3
88      6.3
212     0.0
Name: PTS, Length: 71, dtype: float64>